In [1]:
#Sign Language

In [2]:
def SignLanguageDetection():
    WORKSPACE_PATH = 'Tensorflow/workspace'
    SCRIPTS_PATH = 'Tensorflow/scripts'
    APIMODEL_PATH = 'Tensorflow/models'
    ANNOTATION_PATH = WORKSPACE_PATH + '/annotations'
    IMAGE_PATH = WORKSPACE_PATH + '/images'
    MODEL_PATH = WORKSPACE_PATH + '/models'
    PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models'
    CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
    CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

    labels = [
        {'name': 'hello', 'id': 1},
        {'name': 'thanks', 'id': 2},
        {'name': 'yes', 'id': 3},
        {'name': 'no', 'id': 4},
        {'name': 'i love you', 'id': 5},
        {'name': 'repeat', 'id': 6},
        {'name': 'help', 'id': 7},
        {'name': 'house', 'id': 8},
        {'name': 'what', 'id': 9},
        {'name': 'family', 'id': 10},
        {'name': 'sorry', 'id': 11}
    ]

    with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
        for label in labels:
            f.write('item { \n')
            f.write('\tname:\'{}\'\n'.format(label['name']))
            f.write('\tid:{}\n'.format(label['id']))
            f.write('}\n')

    CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

    import tensorflow as tf
    from object_detection.utils import config_util
    from object_detection.protos import pipeline_pb2
    from google.protobuf import text_format

    CONFIG_PATH = MODEL_PATH + '/' + CUSTOM_MODEL_NAME + '/pipeline.config'

    CONFIG_PATH

    config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

    config

    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
    with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:
        proto_str = f.read()
        text_format.Merge(proto_str, pipeline_config)

    pipeline_config.model.ssd.num_classes = 11
    pipeline_config.train_config.batch_size = 4
    pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH + '/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
    pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
    pipeline_config.train_input_reader.label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
    pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
    pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
    pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

    config_text = text_format.MessageToString(pipeline_config)
    with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:
        f.write(config_text)

    import os  # **
    from object_detection.utils import label_map_util
    from object_detection.utils import visualization_utils as viz_utils
    from object_detection.builders import model_builder

    # Load pipeline config and build a detection model#**
    configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
    detection_model = model_builder.build(model_config=configs['model'], is_training=False)

    # Restore checkpoint
    ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
    ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-11')).expect_partial()

    @tf.function
    def detect_fn(image):
        image, shapes = detection_model.preprocess(image)
        prediction_dict = detection_model.predict(image, shapes)
        detections = detection_model.postprocess(prediction_dict, shapes)
        return detections

    import cv2
    import numpy as np

    category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH + '/label_map.pbtxt')

    # Setup capture
    cap = cv2.VideoCapture(0)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    while True:
        ret, frame = cap.read()
        image_np = np.array(frame)

        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                      for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes'] + label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.5,
            agnostic_mode=False)

        cv2.imshow('Sign Language Detection', cv2.resize(image_np_with_detections, (800, 600)))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            break

    detections = detect_fn(input_tensor)

    from matplotlib import pyplot as plt


# SignLanguageDetection()

In [3]:
############################Voicemail#################3
import smtplib
import speech_recognition as sr
import pyttsx3
# from email.message import EmailMessage
import easyimap as e

unm = 'sahuamisha@kccemsr.edu.in'
pwd = 'Kccemsr@12345'

r = sr.Recognizer()
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voices', voices[1].id)
engine.setProperty('rate', 150)


def speak(str):
    print(str)
    engine.say(str)
    engine.runAndWait()


def listen():
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        str = "Speak Now:"
        speak(str)
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio)
            return text
        except:
            str = "Sorry could not recognize what you said"
            speak(str)


def sendmail():
    rec = 'muskankpandey100@gmail.com'

    str = "Please speak the body of your email"
    speak(str)
    msg = listen()

    str = "You have spoken the message"
    speak(str)
    speak(msg)

    server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
    server.login(unm, pwd)
    server.sendmail(unm, rec, msg)
    server.quit()

    str = "The mail has been sent"
    speak(str)


def readmail():
    server = e.connect("imap.gmail.com", unm, pwd)
    server.listids()

    str = "Please say the Serial Number of the email you wana read starting from latest"
    speak(str)

    a = listen()
    if (a == "Tu"):
        a = "2"

    b = int(a) - 1
    email = server.mail(server.listids()[b])

    str = "The email is from: "
    speak(str)
    speak(email.from_addr)
    str = "The subject of the email is: "
    speak(str)
    speak(email.title)
    str = "The body of email is:"
    speak(str)
    # speak(str)
    speak(email.body)


def voicemail():
    str = "Welcome to voice controlled email service"
    speak(str)

    while (True):
        str = "Speak SEND to Send email   Speak READ to Read Inbox    Speak EXIT to exit"
        speak(str)

        ch = listen()

        if (ch == 'send'):
            str = "You have choosen to send an email"
            speak(str)
            sendmail()

        elif (ch == "read"):
            str = "You have choosen to read the email"
            speak(str)
            readmail()

        elif (ch == "exit"):
            str = "You have choosen to exit,Bye bye"
            speak(str)
            # exit(True)
            break;

        else:
            str = "Invalid choice."
            speak(str)


# voicemail()

In [4]:
########################TKINTER############################
from tkinter import *
from tkinter import messagebox
import tkinter as tk

window = Tk()
window.geometry("744x600")
window.title("SelfAssistance for Disable")


# This was created just for testing for actual demo run the above 2 cells







#lOOGO IMAGE
logoImage = PhotoImage(file=r"D:\python\logo.png")
logoImage=logoImage.subsample(10,10)

#TITLE LABEL
Title1 = Label(window, text="      SelfAssistance for Disable", width=700, height=100,borderwidth=0, image=logoImage,anchor=NW, compound=LEFT,
               fg="#ef7d4c", relief="solid", font=("Open Sans", 20, "bold"))
Title1.place(x=20, y=12)

#VOICEMAIL IMAGE
voiceMailImage = PhotoImage(file=r"D:\python\voicemail.png").subsample(1,2)

#VOICEMAIL BUTTON
VoiceMailButton = Button(window, text="VoiceMail", height=350, width=300, fg="white",bg='#ef7d4c',borderwidth=0,font=("Open Sans", 20, "bold"), image=voiceMailImage,
                         compound=TOP,command=voicemail)  # VoiceMailButton
VoiceMailButton.place(x=400, y=120)

#SIGNLANGUAGE IMAGE
SignLanguageImage = PhotoImage(file=r"D:\python\signlanguage.png").subsample(1,2)

#SIGNLANGUAGE BUTTON
SignLanguageButton = Button(window, text="SignLanguage Detection", width=300, height=350, fg="white",borderwidth=0,bg='#ef7d4c',font=("Open Sans", 18, "bold"),image=SignLanguageImage, compound=TOP,command=SignLanguageDetection)  # command=SignLanguageDetection
SignLanguageButton.pack(pady=20)
SignLanguageButton.place(x=40, y=120)

#EXIT BUTTON
ExitButton = Button(window, text="Exit", width=50, fg="white",bg='#ef7d4c',borderwidth=0, font=("Open Sans", 15, "bold"),command=exit)
ExitButton.pack(pady=30)
ExitButton.place(x=70, y=520)

window.mainloop()



Welcome to voice controlled email service
Speak SEND to Send email   Speak READ to Read Inbox    Speak EXIT to exit
Speak Now:
Sorry could not recognize what you said
Invalid choice.
Speak SEND to Send email   Speak READ to Read Inbox    Speak EXIT to exit
Speak Now:
You have choosen to send an email
Please speak the body of your email
Speak Now:
You have spoken the message
good afternoon
The mail has been sent
Speak SEND to Send email   Speak READ to Read Inbox    Speak EXIT to exit
Speak Now:
Sorry could not recognize what you said
Invalid choice.
Speak SEND to Send email   Speak READ to Read Inbox    Speak EXIT to exit
Speak Now:
You have choosen to read the email
Please say the Serial Number of the email you wana read starting from latest
Speak Now:
The email is from: 
BHARATI SINGH <singhbharati@kccemsr.edu.in>
The subject of the email is: 
This is a test subject
The body of email is:
This is a test message

Speak SEND to Send email   Speak READ to Read Inbox    Speak EXIT to exit

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Ritu\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Ritu\AppData\Local\Temp/ipykernel_10304/3959681233.py", line 101, in SignLanguageDetection
    detections = detect_fn(input_tensor)
  File "C:\Users\Ritu\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\eager\def_function.py", line 885, in __call__
    result = self._call(*args, **kwds)
  File "C:\Users\Ritu\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\eager\def_function.py", line 917, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "C:\Users\Ritu\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\eager\function.py", line 3039, in __call__
    return graph_function._call_flat(
  File "C:\Users\Ritu\AppData\Local\Programs\Python\Python39\lib\site-p